<a href="https://colab.research.google.com/github/Gayatri8-sys/TTS/blob/main/svara_tts_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
print(torch.cuda.get_device_name(0))

Tesla T4


In [ ]:
!nvidia-smi

Sat Nov 29 17:27:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 125.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2


## Local Inference on GPU
Model page: https://huggingface.co/kenpath/svara-tts-v1

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/kenpath/svara-tts-v1)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-to-speech", model="kenpath/svara-tts-v1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("kenpath/svara-tts-v1")
model = AutoModelForCausalLM.from_pretrained("kenpath/svara-tts-v1")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from scipy.io.wavfile import write
import numpy as np

# Define the text to be converted to speech
text = "Hello, this is a test of the Svara TTS model."

# Generate speech audio
output = pipe(text)

# The pipeline output is a dictionary, extract the audio array and sampling rate
# The 'audio' key typically contains the numpy array and 'sampling_rate' for the rate.
# Let's assume the output structure based on common transformers pipeline outputs.
# If this structure causes an error, further inspection of the 'output' variable will be needed.

# Check if the output is a list of dictionaries (common for text-to-speech pipelines)
if isinstance(output, list) and len(output) > 0 and 'audio' in output[0] and 'sampling_rate' in output[0]:
    audio_array = output[0]['audio']
    sampling_rate = output[0]['sampling_rate']
else:
    # Fallback or error handling if the expected structure is not found
    print("Unexpected output format from the pipeline. Please inspect the 'output' variable.")
    # Attempt to use a common alternative structure or raise an error
    # For this example, let's assume a direct dictionary if not a list of one
    if 'audio' in output and 'sampling_rate' in output:
        audio_array = output['audio']
        sampling_rate = output['sampling_rate']
    else:
        raise ValueError("Could not extract audio_array and sampling_rate from pipeline output.")

# Fix: The sampling_rate from the pipeline output is None.
# According to the model card (kenpath/svara-tts-v1), the sampling rate is 22050 Hz.
if sampling_rate is None:
    sampling_rate = 22050

# Normalize audio to 16-bit PCM range if it's float
# Fix: The audio_array already contains large float values (e.g., 1.28e5)
# which are likely meant to be direct 16-bit samples represented as floats.
# Therefore, convert directly to int16 without multiplying by 32767.
if audio_array.dtype == np.float32 or audio_array.dtype == np.float64:
    audio_array = audio_array.astype(np.int16)

# Save the audio to a WAV file
output_filename = "output.wav"
write(output_filename, sampling_rate, audio_array)

print(f"Speech saved to {output_filename}")

Unexpected output format from the pipeline. Please inspect the 'output' variable.
Speech saved to output.wav
